# Introduction

We're going to train a model to detect anomalous electrocardiogram (ECG) signals.

# Dataset


[The PTB Diagnostic ECG Database](https://www.physionet.org/physiobank/database/ptbdb/)

> This dataset has been used in exploring heartbeat classification using deep neural network architectures, and observing some of the capabilities of transfer learning on it. The signals correspond to electrocardiogram (ECG) shapes of heartbeats for the normal case and the cases affected by different arrhythmias and myocardial infarction. These signals are preprocessed and segmented, with each segment corresponding to a heartbeat.
- Number of Samples: 14552
- Number of Categories: 2
- Sampling Frequency: 125Hz
- Data Source: Physionet's PTB Diagnostic Database

**Note:** _All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188._

In [690]:
import altair as alt
import matplotlib as plt
import numpy as np
import polars as pl
import random
import torch

from collections import OrderedDict
from datetime import datetime, timedelta
from pathlib import Path
from tqdm.notebook import tqdm
from ecg import DATA_DIR

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
# The datasets have been downloaded, converted to .parquet, and moved to the `data/` directory
train_file = "mitbih_train.parquet"
test_file = "mitbih_test.parquet"

train_df = pl.read_parquet(DATA_DIR / train_file)
test_df = pl.read_parquet(DATA_DIR / test_file)

In [ ]:
abnormal_file = "ptbdb_abnormal.parquet"
normal_file = "ptbdb_normal.parquet"

abnormal_df = pl.read_parquet(DATA_DIR / abnormal_file)
normal_df = pl.read_parquet(DATA_DIR / normal_file)

## EDA

In [ ]:
abnormal_df.shape, normal_df.shape

In [ ]:
# normal_df = normal_df.with_columns(pl.Series("target", ["normal"] * len(normal_df)))
# abnormal_df = abnormal_df.with_columns(pl.Series("target", ["abnormal"] * len(normal_df)))


df = pl.concat(
    [
        normal_df.with_columns(pl.Series("class", ["normal"] * normal_df.shape[0])),
        abnormal_df.with_columns(pl.Series("class", ["abnormal"] * abnormal_df.shape[0]))
    ],
    how="vertical"
)

In [ ]:
(
    df.filter(
        pl.col("class") == "normal"
    )
    .drop(["class", "target"])
    .sample(n=2) # we don't seed so that we can see different samples
    # .transpose()
)

In [ ]:
# def plot_samples(df: pl.DataFrame, num_samples: int, case: str) -> alt.Chart:
#     plots = []
#     for idx in samples:
#         plot = (
#             df.filter(
#                 pl.col("class") == class_name
#             )
#             .drop("class")
#             .with_row_index()
#             .select(["index", f"column_{idx}"])
#             .rename({f"column_{idx}": "signal"})
#             .with_columns(pl.Series("case", [idx] * len(df)))
#         )
#         plots.append(plot)

#     return pl.concat(
#         plots,
#         how="vertical",
#     )

def plot_samples(df: pl.DataFrame, class_name: str, samples: int = 100, opacity: float = 0.05) -> alt.Chart:
    data = (
        df.filter(
            pl.col("class") == class_name
        )
        .drop(["class", "target"])
        .sample(n=samples) # we don't seed so that we can see different samples
        .transpose()
    )
    plot_df = (
        pl.concat([
            data
            .select(col)
            .rename({col: "signal"})
            .with_columns(
                pl.Series("case", [i] * data.shape[0]),
                pl.Series("measurement", list(range(data.shape[0]))),
                pl.Series("color", [0] * data.shape[0]),
            )
            for i, col in enumerate(data.columns)
        ], 
        how="vertical")
        .with_row_index()
    )
    
    return alt.Chart(plot_df, title=class_name).mark_line().encode(
        x=alt.X("measurement", title="measurement"),
        y=alt.Y("signal", title="signal", scale=alt.Scale(domain=[0.0, 1.2])),
        color=alt.Color("color:N", title=None),
        detail="case",
        opacity=alt.value(opacity),
    ).properties(height=300, width=400)

In [ ]:
plot_samples(df, "normal", 100, 0.1) | plot_samples(df, "abnormal", 100, 0.1) 

We can see that signals for both the "normal" cases and "abnormal" are between [0.0, 1.0]  and all cases start at or near 1.0.  
From the 75th measurement (0.6 seconds from the start) the signals can spike to near 1.0 for a couple time steps before dropping abck to nominal levels.
Some cases drop to 0.0 from about the 100th measurement (0.8 seconds from the start), this just means the signal ended early and has been padded with zeros until the 188th measurement.

The normal cases have a spike in the signal at about the 35th measurement (0.28 seconds from the start) spread across ~30 measurements (0.24 seconds).  Those that don't end early tend to have another spike at the end spread across ~30 measurements.

The abnormal cases are more variable, especially around where the first spike occurs in the normal cases.  The nominal level for each case is also more variable.

Let's plot the mean and standard deviation across each of the classes.

In [ ]:
def plot_rolling_mean(df: pl.DataFrame, class_name: str, window: int = 5) -> alt.Chart:
    period = f"{window}i"
    
    data = df.filter(pl.col("class") == class_name).drop("class")
    mean = data.mean()
    std = data.std()
    
    rolling_mean_df = (
        mean.drop("target").transpose()
        .with_row_index()
        .rolling("index", period=period)
        .agg([
            pl.col(pl.Float64).mean()
        ])
        .drop("index")
    )
    
    rolling_std_df = (
        std.drop("target").transpose()
        .with_row_index()
        .rolling("index", period=period)
        .agg([
            pl.col(pl.Float64).mean()
        ])
        .drop("index")
    )
    margin = rolling_std_df * 2

    # signals are always positive so we clip the low
    lower_bound = (rolling_mean_df - margin).with_columns(pl.col(pl.Float64).clip(0.0, 1.0))
    upper_bound = (rolling_mean_df + margin).with_columns(pl.col(pl.Float64).clip(0.0, 1.0))

    plot_df = pl.concat([
        rolling_mean_df.rename({"column_0": "mean"}),
        lower_bound.rename({"column_0": "lower"}),
        upper_bound.rename({"column_0": "upper"})
    ], how="horizontal").with_row_index()
    
    line = alt.Chart(plot_df).mark_line().encode(
        x=alt.X("index", title="measurement"),
        y=alt.Y("mean", title="mean signal")
    )
    
    band = alt.Chart(plot_df, title=class_name).mark_area().encode(
        x=alt.X("index", title="measurement"),
        y=alt.Y("lower", scale=alt.Scale(domain=[-0.1, 1.1])),
        y2=alt.Y2("upper"),
        opacity=alt.value(0.25),
    ).properties(height=300, width=400)
    
    return band + line

In [ ]:
plot_rolling_mean(df, "normal", 5) | plot_rolling_mean(df, "abnormal", 5)

Here we've calculated the mean and standard deviation for each class at each measurement then taken a rolling average over the given window to smooth out the curve.
The doubled standard deviation is shown as the upper and lower bounds, clipped to the range [1.0, 0.0].

In the normal cases, we see a distinct spike at the 30th measurement and another more spread out spike about the 110th measurement.  The nominal signal level is ~0.2.  The signals end at about the 120th measurement (nominal level starts to decrease).

In the abnormal cases, we see the same spikes but with less prominence, meaning when the spike occurs is variable.  The second spike is notably shifted forward to about the 90th measurement.  Overall, the signal has a much higher spread and the nominal signal level is ~0.25-0.3.   The signals tend to end 20 measurements earlier than normal signals, at the 100th measurement (nominal level starts to decrease).

## Modelling

In [681]:
class ECGDataset(torch.utils.data.Dataset):   
    """Dataset to sample cases from the training dataset.

    TODO: randomly sample variable lengths
    """
    def __init__(self, X: pl.DataFrame, y: pl.Series) -> None:
        self.data = X
        self.target = y
        self.X = X.to_torch().to(torch.float32)
        self.y = y.to_torch().to(torch.float32)

    def __len__(self) -> None:
        return len(self.X)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.X[idx]

    def get_row(self, idx: int) -> pl.DataFrame:
        return pl.concat([self.target, self.data], how="horizontal")

In [691]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvAutoEncoder(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int) -> None:
        super(ConvAutoEncoder, self).__init__()
        self.input_dim = input_dim # (188)
        self.hidden_dim = hidden_dim

        # Building an linear encoder with Linear layer followed by Relu activation function
        # 188 ==> 16
        self.encoder = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(self.input_dim, 128)),
            ('relu1', nn.ReLU()),
            ('linear2', nn.Linear(128, 64)),
            ('relu2', nn.ReLU()),
            ('linear3', nn.Linear(64, 32)),
            ('relu3', nn.ReLU()),
            ('linear4', nn.Linear(32, 16)),
        ]))

        # Building an linear decoder with Linear layer followed by Relu activation function
        # The Sigmoid activation function outputs the value between 0 and 1
        # 16 ==> 188
        self.decoder = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(16, 32)),
            ('relu1', nn.ReLU()),
            ('linear2', nn.Linear(32, 64)),
            ('relu2', nn.ReLU()),
            ('linear3', nn.Linear(64, 128)),
            ('relu3', nn.ReLU()),
            ('linear4', nn.Linear(128, self.input_dim)),
            ("activation", torch.nn.Sigmoid())
        ]))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        encoded = self.encoder(x) 
        decoded = self.decoder(encoded)
        return decoded
        
class ConvAutoEncoder(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int, stride: int) -> None:
        super(ConvAutoEncoder, self).__init__()
        self.input_dim = input_dim # (188)
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.stride = 1

        # Building an linear encoder with Linear layer followed by Relu activation function
        # 188 ==> 16

        assert (self.kernel_size % 2 != 0) # and (stride == 1)
        pool_padding = (kernel_size - 1) // 2
        
        self.encoder = nn.Sequential(OrderedDict([
            # Conv1d Layer 1: Input Channels = 1, Output Channels = hidden_features
            ('conv1', nn.Conv1d(1, self.hidden_dim, kernel_size=kernel_size, stride=self.stride)),
            ('relu1', nn.ReLU()),
            ('norm1', nn.BatchNorm1d(self.hidden_dim)),
            ('pool1', nn.MaxPool1d(kernel_size=kernel_size, stride=1, padding=pool_padding)),

            # # Conv1d Layer 2: Input Channels = hidden_features, Output Channels = hidden_features
            # ('conv2', nn.Conv1d(self.hidden_dim, self.hidden_dim, kernel_size=kernel_size, stride=self.stride)),
            # ('relu2', nn.ReLU()),
            # ('norm2', nn.BatchNorm1d(self.hidden_dim)),
            # ('pool2', nn.MaxPool1d(kernel_size=kernel_size, stride=1, padding=pool_padding)),

            # # Conv1d Layer 2: Input Channels = hidden_features, Output Channels = hidden_features
            # ('conv3', nn.Conv1d(self.hidden_dim, self.hidden_dim, kernel_size=kernel_size, stride=self.stride)),
            # ('relu3', nn.ReLU()),
            # ('norm3', nn.BatchNorm1d(self.hidden_dim)),
            # ('pool3', nn.MaxPool1d(kernel_size=kernel_size, stride=1, padding=pool_padding)),
        ]))

        # Building an linear decoder with Linear layer followed by Relu activation function
        # The Sigmoid activation function outputs the value between 0 and 1
        # 16 ==> 188
        self.decoder = nn.Sequential(OrderedDict([
            # # Conv1d Layer 1: Input Channels = hidden_features, Output Channels = hidden_features
            # ('conv1', nn.ConvTranspose1d(self.hidden_dim, self.hidden_dim, kernel_size=kernel_size, stride=self.stride)),
            # ('relu1', nn.ReLU()),
            # ('norm1', nn.BatchNorm1d(self.hidden_dim)),

            # # Conv1d Layer 1: Input Channels = hidden_features, Output Channels = hidden_features
            # ('conv2', nn.ConvTranspose1d(self.hidden_dim, self.hidden_dim, kernel_size=kernel_size, stride=self.stride)),
            # ('relu2', nn.ReLU()),
            # ('norm2', nn.BatchNorm1d(self.hidden_dim)),

            # Conv1d Layer 1: Input Channels = hidden_features, Output Channels = 1
            ('conv3', nn.ConvTranspose1d(self.hidden_dim, 1, kernel_size=kernel_size, stride=self.stride)),
            ('relu3', nn.ReLU()),
            ('norm3', nn.BatchNorm1d(1)),
        ]))

    def forward(self, x: torch.Tensor):
        # our input is (n_batch, length_sequence) we need to add a dimension for n_channels (n_batch, n_channels, length_sequence)
        # for the Conv1D layers
        encoded = self.encoder(x[:, None, :]) 
        decoded = self.decoder(encoded)
        return decoded.squeeze()

In [702]:
input_dim = train_dataset[0].shape[0]
hidden_dim = 128
batch_size = 32
kernel_size = 9
stride = 1

model = ConvAutoEncoder(input_dim, hidden_dim, kernel_size, stride)
# model = torch.compile(model)

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()
 
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = 1e-1,
    weight_decay = 1e-8
)
# scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer, 
#     step_size=4, 
#     gamma=0.1, 
# )
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode="min",
    patience=2
)

dataset = ECGDataset(df.drop(["target", "class"]), df.select("target"))
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

gen = torch.Generator()
gen.manual_seed(0)

train_dataloader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    worker_init_fn=seed_worker,
    generator=gen,
    # num_workers=4,
    # persistent_workers=True
)

val_dataloader = torch.utils.data.DataLoader(
    dataset = val_dataset,
    batch_size = batch_size,
    shuffle = True,
    worker_init_fn=seed_worker,
    generator=gen,
    # num_workers=4,
    # persistent_workers=True
)

In [703]:
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)

epochs = 20
outputs = []
train_batch_losses = []
train_epoch_losses = []
val_batch_losses = []
val_epoch_losses = []

for epoch in (pbar := tqdm(range(epochs), desc=f"lr={scheduler.get_last_lr()[0]:0.6f}")):
    ####################
    # Trian
    ####################
    model.train()
    train_epoch_loss = 0
    for i, batch in enumerate(train_dataloader):
        
        # Output of Autoencoder
        reconstructed = model(batch)
        
        # Calculating the loss function
        loss = loss_function(reconstructed, batch)
        
        optimizer.zero_grad() # The gradients are set to zero,
        loss.backward() # the gradients are computed and stored.
        optimizer.step() # .step() performs parameter update
        
        # Storing the losses in a list for plotting
        train_batch_losses.append(loss.item())
        train_epoch_loss += loss.item()

    train_mean_epoch_loss = train_epoch_loss / i
    train_epoch_losses.append(train_mean_epoch_loss)
    
    ####################
    # Validation
    ####################
    model.eval()
    with torch.no_grad(): 
        val_epoch_loss = 0
        for j, batch in enumerate(train_dataloader):
            reconstructed = model(batch)            
            loss = loss_function(reconstructed, batch)
       
            val_batch_losses.append(loss.item())
            val_epoch_loss += loss.item()

        val_mean_epoch_loss = val_epoch_loss / j
        val_epoch_losses.append(val_mean_epoch_loss)

    # scheduler.step()
    scheduler.step(val_mean_epoch_loss) # update the learning rate if not learning
    lr = scheduler.get_last_lr()[0]
    pbar.set_description(f"lr={lr:0.6f}")
    print(f"{epoch} (lr={lr:0.6f}) - train loss: {train_mean_epoch_loss} | val loss: {val_mean_epoch_loss}")
    
    # outputs.append((epochs, batch, reconstructed))


lr=0.100000:   0%|          | 0/20 [00:00<?, ?it/s]

0 (lr=0.100000) - train loss: 0.014709900274741932 | val loss: 0.0014465140520669418
1 (lr=0.100000) - train loss: 0.002493925500384047 | val loss: 0.0010497346929320681
2 (lr=0.100000) - train loss: 0.0021300837145823235 | val loss: 0.0032221315577420978
3 (lr=0.100000) - train loss: 0.0020435197516498423 | val loss: 0.001457369564031828
4 (lr=0.100000) - train loss: 0.0023938551184031136 | val loss: 0.0009478018109201769
5 (lr=0.100000) - train loss: 0.002292006290177802 | val loss: 0.006345001736927788
6 (lr=0.100000) - train loss: 0.0019920289937532492 | val loss: 0.0009621792256744892
7 (lr=0.100000) - train loss: 0.0020244293629715204 | val loss: 0.0006628400810029532
8 (lr=0.100000) - train loss: 0.0021632876934755602 | val loss: 0.0008716777707232014
9 (lr=0.100000) - train loss: 0.0020279439835496167 | val loss: 0.0017379648985402767
10 (lr=0.010000) - train loss: 0.002221767469270786 | val loss: 0.0008392722221455709
11 (lr=0.010000) - train loss: 0.0012259723193854986 | val 

In [704]:
batch_losses_df = pl.DataFrame({
    "batch": [x for x in range(0, len(train_batch_losses))] + [x for x in range(0, len(val_batch_losses))],
    "loss": train_batch_losses + val_batch_losses,
    "mode": ["train"] * len(train_batch_losses) + ["validation"] * len(val_batch_losses)
})

epoch_losses_df = pl.DataFrame({
    "epoch": [x for x in range(0, len(train_epoch_losses))] + [x for x in range(0, len(val_epoch_losses))],
    "loss": train_epoch_losses + val_epoch_losses,
    "mode": ["train"] * len(train_epoch_losses) + ["validation"] * len(val_epoch_losses)
})

batch_loss_chart = (
    alt.Chart(batch_losses_df.to_pandas(), title="Loss per Batch")
    .mark_line()
    .encode(
        x='batch:Q',
        y='loss:Q',
        color='mode:N',
    ).properties(height=300, width=400)
)

epoch_loss_chart = (
    alt.Chart(epoch_losses_df.to_pandas(), title="Average Loss per Epoch")
    .mark_line()
    .encode(
        x='epoch:Q',
        y='loss:Q',
        color='mode:N',
    ).properties(height=300, width=400)
)

batch_loss_chart | epoch_loss_chart

alt.HConcatChart(...)

# Scratch Pad

In [ ]:
# [nBatch, nChannels, length]
hidden_features = 128
# stride need to be 1 and kernel_size uneven for the padding to work and dimensions align
kernel_size = 9

# encode
print("Encoding:")
m = nn.Conv1d(1, hidden_features, kernel_size=kernel_size, stride=1)
input = torch.randn(32, 1, 187)
output = m(input)
print("Conv:     ", input.shape, output.shape)

norm_input = output
norm = nn.BatchNorm1d(hidden_features)
norm_output = norm(norm_input)
print("Norm:     ", norm_input.shape, norm_output.shape)

pool_input = norm_output
pool = nn.MaxPool1d(kernel_size=kernel_size, stride=1, padding=(kernel_size-1)//2) # (kernel_size-1)//2 - https://stackoverflow.com/a/71022586
pool_output = pool(pool_input)
print("Pool:     ", pool_input.shape, pool_output.shape)


# decode
print("\nDecoding:")
conv_input = pool_output
conv_input = norm_output
conv = nn.ConvTranspose1d(hidden_features, 1, kernel_size=kernel_size, stride=1)
conv_output = conv(conv_input)
print("ConvTrans:", conv_input.shape, conv_output.shape)

norm_input = conv_output
norm = nn.BatchNorm1d(1)
norm_output = norm(norm_input)
print("Norm:     ", norm_input.shape, norm_output.shape)


# linear_input = norm_output
# linear = nn.Linear(32, hidden_features, 187)
# linear_output = linear(linear_input)
# print("Linear:   ", linear_input.shape, linear_output.shape)

In [ ]:
a = torch.zeros(4, 5, 6)
a = a[:, :, None, :]
a.shape, a.squeeze().shape

In [ ]:
torch.randn(32, 187).view(32, 1, 187).shape

In [ ]:
(kernel_size-1)//2

In [682]:
dataset = ECGDataset(df.drop(["target", "class"]), df.select("target"))
dataset.get_row(0)

target,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_151,column_152,column_153,column_154,column_155,column_156,column_157,column_158,column_159,column_160,column_161,column_162,column_163,column_164,column_165,column_166,column_167,column_168,column_169,column_170,column_171,column_172,column_173,column_174,column_175,column_176,column_177,column_178,column_179,column_180,column_181,column_182,column_183,column_184,column_185,column_186,column_187
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,1.0,0.900324,0.35859,0.051459,0.046596,0.126823,0.133306,0.119125,0.110616,0.113047,0.106564,0.106969,0.115883,0.122366,0.122366,0.11953,0.115883,0.122366,0.126013,0.133712,0.134927,0.142626,0.151135,0.158428,0.163695,0.173825,0.188817,0.207861,0.230956,0.258509,0.294571,0.32577,0.362642,0.398298,0.429498,0.449352,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.794681,0.375387,0.116883,0.0,0.171923,0.283859,0.293754,0.325912,0.345083,0.361781,0.3624,0.36611,0.367965,0.37415,0.37786,0.382189,0.384663,0.398887,0.401361,0.418058,0.443414,0.457638,0.487941,0.520717,0.559678,0.604205,0.634508,0.65368,0.672851,0.678417,0.660482,0.621521,0.555968,0.482375,0.438466,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.909029,0.791482,0.423169,0.186712,0.0,0.007836,0.063032,0.077002,0.074957,0.077342,0.077342,0.087223,0.091993,0.09506,0.096422,0.10494,0.108007,0.113799,0.116525,0.119932,0.124361,0.132198,0.145826,0.152641,0.163543,0.175468,0.189438,0.20477,0.229302,0.252811,0.27598,0.302555,0.321295,0.333901,0.345826,0.348552,…,0.174106,0.178194,0.183646,0.186371,0.191141,0.194549,0.201704,0.208177,0.212266,0.219762,0.227598,0.238842,0.249063,0.254514,0.270187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.478893,0.05676,0.064176,0.081289,0.072732,0.055619,0.048774,0.054478,0.041643,0.049059,0.051341,0.049344,0.045921,0.049914,0.053908,0.049629,0.045921,0.055048,0.053622,0.063605,0.066172,0.083286,0.09498,0.115801,0.131204,0.153166,0.178266,0.210211,0.232744,0.266115,0.286651,0.305762,0.317456,0.304906,0.264689,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.867238,0.20136,0.099349,0.141336,0.120934,0.108516,0.096393,0.093436,0.100828,0.086931,0.094027,0.095801,0.096393,0.089592,0.094914,0.089592,0.094914,0.098167,0.102306,0.099645,0.096688,0.108516,0.127735,0.128326,0.149024,0.172383,0.204021,0.221762,0.254287,0.284743,0.309284,0.329687,0.344175,0.359846,0.33353,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1.0,0.981409,1.0,0.559171,0.287093,0.196639,0.204862,0.215946,0.243833,0.24276,0.250268,0.254201,0.253843,0.253128,0.258849,0.263139,0.257419,0.27029,0.265284,0.267787,0.281373,0.273865,0.286378,0.297462,0.294244,0.313908,0.324991,0.338935,0.343225,0.369682,0.38291,0.390061,0.402932,0.393279,0.4133,0.396139,0.387201,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0